# Libraries


In [ ]:
# Import necessary libraries for building and evaluating a neural network
import os
import numpy as np
import joblib
import matplotlib.pyplot as plt
import librosa


# Layers from Keras (for defining the model architecture)
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, Activation

# Sequential model class from Keras (to build the model layer by layer)
from tensorflow.keras.models import Sequential

from tensorflow.keras.optimizers import Adam

# Evaluation tools from Scikit-learn (to assess model performance)
from sklearn.metrics import confusion_matrix, classification_report

# Data splitting tool from Scikit-learn (to split the data into training and testing sets)
from sklearn.model_selection import train_test_split



___
Testing Audio Files

from IPython.display import Audio, display
a_path = '/home/badri/B/ser/datasets/revdess/Actor_01/03-01-01-01-01-01-01.wav'
a_path = '/home/badri/B/ser/datasets/neutral.wav'
sr = 22050
display(Audio(filename=a_path, rate = sr))
a_path = '/home/badri/B/ser/datasets/sad_audio.wav'

y,sam_r = librosa.load(a_path, sr = sr)

plt.figure(figsize=(14,5))
librosa.display.waveshow(y, sr = sr)
plt.title("testing_wave_form")
plt.show()

end
___


# Feature Extraction


%%time


def extract_mfcc(file_path, sr = 22050, n_mfcc = 40):
    y,sr = librosa.load(file_path, sr=sr, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    return mfccs_mean

dataset_path = '/home/badri/B/ser/datasets/revdess/'

mfcc_features = []

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith('.wav'):
            try:
                file_path = os.path.join(root, file)
                mfccs = extract_mfcc(file_path)
                file_class = int(file[7:8]) -1 
                mfcc_features.append((mfccs, file_class))
            except ValueError as err:
                print(f"Error processing file {file}: {err}")
                continue
                
print(f" Extracted {len(mfcc_features)}files")
for  i in range(10):
    print(mfcc_features[i])
    
    


# saving Features 

%%time


x, y = zip(*mfcc_features)
x, y = np.asarray(x), np.asarray(y)
print(x.shape, y.shape)

save_dir_path = '/home/badri/B/ser/datasets/MFCC_Features'

joblib.dump(x, os.path.join(save_dir_path, 'x.joblib'))
joblib.dump(y, os.path.join(save_dir_path, 'y.joblib'))

In [ ]:
x = joblib.load('/home/badri/B/ser/datasets/MFCC_Features/x.joblib')
y = joblib.load('/home/badri/B/ser/datasets/MFCC_Features/y.joblib')



In [ ]:
%%time
x_train,  x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

x_traincnn = np.expand_dims(x_train, axis = 2)
x_testcnn = np.expand_dims(x_test, axis = 2)
print(x_traincnn.shape, x_testcnn.shape)

# Model Architecture

In [ ]:
model = Sequential()
model.add(Conv1D(64, 5, padding='same', input_shape=(40, 1)))  # Conv1D layer
model.add(Activation('relu'))  # Activation layer
model.add(Dropout(0.2))  # Dropout layer
model.add(Flatten())  # Flatten layer
model.add(Dense(8))  # Dense layer
model.add(Activation('softmax'))  # Activation layer
model.summary()


# Model Training

%%time
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(), loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_traincnn, y_train, epochs=70, batch_size=32, validation_data=(x_testcnn, y_test))

y_pred = model.predict(x_testcnn)
y_pred_classes = np.argmax(y_pred, axis =1)
print(confusion_matrix(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))

plt.plot(history.history['accuracy'], label = 'accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel("accuracy")
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label = 'loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel("loss")

plt.legend(loc='upper right')
plt.show()

# save model 

model.save('/home/badri/B/ser/saved_models/model291220241521.keras')

# Load saved model

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/home/badri/B/ser/saved_models/model291220241521.keras')
model.summary()

# Methods def

In [ ]:

def extract_mfcc(file_path, sr = 22050, n_mfcc = 40):
    y,sr = librosa.load(file_path, sr=sr, res_type='kaiser_fast')
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs.T, axis=0)
    return mfccs_mean

def audio_input(audio_path):
    mfccs = extract_mfcc(audio_path)
    new_mfccs = np.expand_dims(mfccs, axis = 0)
    new_mfccs = np.expand_dims(new_mfccs, axis = -1)
    return new_mfccs
    
    

In [ ]:
y_pred = model.predict(audio_input('/home/badri/B/ser/datasets/Nappy.wav'))
y_pred_class = np.argmax(y_pred, axis = 1)

# Feedback Mechanism

In [ ]:
# Still thinking wheather to use joblib or json -sqlite3 or csv/ txt for storing the feedbacks.


from IPython.display import Audio, display
DEF_REQ = 50 # Default Request interval for feedbacks is 50 predictions.

threashold = DEF_REQ

current_score = "current going prediction from 0."

def req_feedback(input_data, my_pred):
    my_current_score = 0
    '''This method requsts feedback about its prediction by  providing the audio to user.'''
    display(Audio(input_data, rate = 22050))
    print(f"I predicted it as: {my_pred}")
    is_correct = input("Am I correct Y?N: ").strip().lower()
    if is_correct == 'y':
        threshold *= 2
        return False
    elif is_correct == 'n':
        user_label = input("please provide me correct label then..:").strip().lower()
        feedback = input_data, user_label, my_pred
        threshold //= 2
        return feedback
    else:
        print("Not a valid feedback! Aborting....feedback.")
        
    
          
def is_idle():
    '''Need to find  out wheather system is idle or not.'''
        
def fine_tuning():
    if is_idle():
        feedbacks = fetch_feedbacks()
        optimizer = Adam
        if len(feedbacks) == 1:
            optimizer.learning_rate = 0.01 # Learnig rate
        else:
            optimizer.learning_rate = 0.001
        model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])
        model.fit(x_data, y_data, batch_size = len(feedbacks), epochs = 1, verbose = 0)
        # need to handle exceptions
        # clear the feedbacks storage for new feedbacks.
        
        
        
        

        

    
    



2